In [1]:
import os 
import sys 
import argparse 
import _pickle as pickle 

import cv2
import numpy as np

from sklearn.cluster import KMeans 
from sklearn.multiclass import OneVsOneClassifier 
from sklearn.svm import LinearSVC 
from sklearn import preprocessing 

In [2]:
class DenseDetector(): 
    def __init__(self, step_size=20, feature_scale=20, img_bound=20): 
        # Create a dense feature detector 
        self.initXyStep = step_size
        self.initFeatureScale = feature_scale
        self.initImgBound = img_bound
 
    def detect(self, img):
        keypoints = []
        rows, cols = img.shape[:2]
        for x in range(self.initImgBound, rows, self.initFeatureScale):
            for y in range(self.initImgBound, cols, self.initFeatureScale):
                keypoints.append(cv2.KeyPoint(float(x), float(y), self.initXyStep))
        return keypoints 
class SIFTExtractor():
    def __init__(self):
        self.extractor = cv2.xfeatures2d.SIFT_create()
       #self.extractor = cv2.SIFT_create()

    def compute(self, image, kps): 
        if image is None: 
            print("Not a valid image")
            raise TypeError 
 
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
        kps, des = self.extractor.detectAndCompute(gray_image, None) 
        return kps, des 

In [3]:
# Vector quantization 
class Quantizer(object): 
    def __init__(self, num_clusters=32): 
        self.num_dims = 128 
        self.extractor = SIFTExtractor() 
        self.num_clusters = num_clusters 
        self.num_retries = 10 
 
    def quantize(self, datapoints): 
        # Create KMeans object 
        kmeans = KMeans(self.num_clusters, 
                        n_init=max(self.num_retries, 1), 
                        max_iter=10, tol=1.0) 
 
        # Run KMeans on the datapoints 
        res = kmeans.fit(datapoints) 
 
        # Extract the centroids of those clusters 
        centroids = res.cluster_centers_
 
        return kmeans, centroids 
 
    def normalize(self, input_data): 
        sum_input = np.sum(input_data) 
        if sum_input > 0: 
            return input_data / sum_input 
        else: 
            return input_data 
 
    # Extract feature vector from the image 
    def get_feature_vector(self, img, kmeans, centroids): 
        kps = DenseDetector().detect(img) 
        kps, fvs = self.extractor.compute(img, kps) 
        labels = kmeans.predict(fvs.astype(float)) 
        fv = np.zeros(self.num_clusters) 
 
        for i, item in enumerate(fvs): 
            fv[labels[i]] += 1 
 
        fv_image = np.reshape(fv, ((1, fv.shape[0]))) 
        return self.normalize(fv_image)

In [4]:
class FeatureExtractor(object): 
    def extract_image_features(self, img): 
        # Dense feature detector 
        kps = DenseDetector().detect(img) 
 
        # SIFT feature extractor 
        kps, fvs = SIFTExtractor().compute(img, kps) 
 
        return fvs 
 
    # Extract the centroids from the feature points 
    def get_centroids(self, input_map, num_samples_to_fit=10): 
        kps_all = [] 
 
        count = 0 
        cur_label = '' 
        for item in input_map: 
            if count >= num_samples_to_fit: 
                if cur_label != item['label']: 
                    count = 0 
                else: 
                    continue 
 
            count += 1 
 
            if count == num_samples_to_fit: 
                print("Built centroids for", item['label'])
 
            cur_label = item['label'] 
            img = cv2.imread(item['image']) 
            img = resize_to_size(img, 150) 
 
            num_dims = 128 
            fvs = self.extract_image_features(img) 
            kps_all.extend(fvs) 
 
        kmeans, centroids = Quantizer().quantize(kps_all) 
        return kmeans, centroids 
 
    def get_feature_vector(self, img, kmeans, centroids): 
        return Quantizer().get_feature_vector(img, kmeans, centroids) 

In [5]:
# Resize the shorter dimension to 'new_size' 
# while maintaining the aspect ratio 
def resize_to_size(input_image, new_size=150): 
    h, w = input_image.shape[0], input_image.shape[1] 
    ds_factor = new_size / float(h) 
 
    if w < h: 
        ds_factor = new_size / float(w) 
 
    new_size = (int(w * ds_factor), int(h * ds_factor)) 
    return cv2.resize(input_image, new_size) 
# Loading the images from the input folder 
def load_input_map(label, input_folder): 
    combined_data = [] 
 
    if not os.path.isdir(input_folder): 
        raise IOError("The folder " + input_folder + " doesn't exist") 
 
    # Parse the input folder and assign the  labels 
    for root, dirs, files in os.walk(input_folder): 
        for filename in (x for x in files if x.endswith('.jpg')): 
            combined_data.append({'label': label, 'image': 
             os.path.join(root, filename)}) 
 
    return combined_data 
def extract_feature_map(input_map, kmeans, centroids): 
    feature_map = [] 
 
    for item in input_map: 
        temp_dict = {} 
        temp_dict['label'] = item['label'] 
 
        print("Extracting features for", item['image'])
        img = cv2.imread(item['image']) 
        img = resize_to_size(img, 150) 
 
        temp_dict['feature_vector'] = FeatureExtractor().get_feature_vector(img, kmeans, centroids) 
 
        if temp_dict['feature_vector'] is not None: 
            feature_map.append(temp_dict) 
 
    return feature_map 

In [6]:
input_map = []

cls = [
        ['Latas de conserva', 'Imagenes depuradas/downloads/latas de conserva'],
        ['Latas de refresco', 'Imagenes depuradas/downloads/latas de refrescos'],
        ['Recipientes PVC', 'Imagenes depuradas/downloads/recipientes pvc'],
        ['Carton', 'Imagenes depuradas/downloads/basura carton'],
        ['Cascara de huevo', 'Imagenes depuradas/downloads/Basura cascara de huevo'],
        ['Envoltorio de frituras', 'Imagenes depuradas/downloads/basura de envolturas de frituras'],
        ['Huesos', 'Imagenes depuradas/downloads/basura de huesos'],
        ['Unicel', 'Imagenes depuradas/downloads/basura de unicel'],
        ['Filtro de cafe', 'Imagenes depuradas/downloads/basura filtro de papel de cafe'],
        ['Metal', 'Imagenes depuradas/downloads/basura metal'],
        ['Plastico', 'Imagenes depuradas/downloads/basura plastico'],
        ['Ropa', 'Imagenes depuradas/downloads/basura ropa'],
        ['Vidrio', 'Imagenes depuradas/downloads/basura vidrio'],
        ['Bolsas usadas', 'Imagenes depuradas/downloads/bolsas de platico usadas'],
        ['Bolsas de te', 'Imagenes depuradas/downloads/bolsas de te usadas'],
        ['Cascara de frutas', 'Imagenes depuradas/downloads/cascaras de fruta'],
        ['Cascara de frutos secos', 'Imagenes depuradas/downloads/cascaras de frutos secos'],
        ['Cascara de verduras', 'Imagenes depuradas/downloads/cascaras de verdura'],
        ['Chicles usados', 'Imagenes depuradas/downloads/chicles usados'],
        ['Colillas de cigarros', 'Imagenes depuradas/downloads/colillas de cigarro'],
        ['Madera', 'Imagenes depuradas/downloads/desechos madera'],
        ['Desperdicio de pescado', 'Imagenes depuradas/downloads/desperdicio de pescado'],
        ['Tetrapack', 'Imagenes depuradas/downloads/envases de tetrapack usados'],
        ['Envolturas metalicas', 'Imagenes depuradas/downloads/envolturas metalicas'],
        ['Jeringas', 'Imagenes depuradas/downloads/jeringas de comida basura'],
        ['Papel', 'Imagenes depuradas/downloads/papel basura'],
        ['Platos desechables', 'Imagenes depuradas/downloads/platos desechables usados'],
        ['Restos de Carne', 'Imagenes depuradas/downloads/restos de carne'],
        ['Restos de Comida', 'Imagenes depuradas/downloads/restos de comida'],
        ['Restos de Pan', 'Imagenes depuradas/downloads/restos de pan'],
        ['Restos de Tortilla', 'Imagenes depuradas/downloads/restos de tortillas'],
        ['Servilletas', 'Imagenes depuradas/downloads/servilletas usadas'],
        ['Vajillas de carton', 'Imagenes depuradas/downloads/vajillas de carton usados']
    
]

for cl in cls: 
    label = cl[0]
    input_map += load_input_map(label, cl[1]) 

In [7]:
kmeans, centroids = FeatureExtractor().get_centroids(input_map) 
with open('codebook_file', 'wb') as f: 
    print('kmeans', kmeans)
    print('centroids', centroids)
    pickle.dump((kmeans, centroids), f)

feature_map = extract_feature_map(input_map, kmeans, 
     centroids) 

with open('feature_map_file', 'wb') as f: 
    pickle.dump(feature_map, f)

Built centroids for Latas de conserva
Built centroids for Latas de refresco
Built centroids for Recipientes PVC
Built centroids for Carton
Built centroids for Cascara de huevo
Built centroids for Envoltorio de frituras
Built centroids for Huesos
Built centroids for Unicel
Built centroids for Filtro de cafe
Built centroids for Metal
Built centroids for Plastico
Built centroids for Ropa
Built centroids for Vidrio
Built centroids for Bolsas usadas
Built centroids for Bolsas de te
Built centroids for Cascara de frutos secos
Built centroids for Cascara de verduras
Built centroids for Chicles usados
Built centroids for Colillas de cigarros
Built centroids for Madera
Built centroids for Desperdicio de pescado
Built centroids for Tetrapack
Built centroids for Envolturas metalicas
Built centroids for Jeringas
Built centroids for Papel
Built centroids for Platos desechables
Built centroids for Restos de Carne
Built centroids for Restos de Comida
Built centroids for Restos de Pan
Built centroids 

Extracting features for Imagenes depuradas/downloads/recipientes pvc\9.hdpe-botellas.jpg
Extracting features for Imagenes depuradas/downloads/basura carton\1.53715146-cajas-de-cart%c3%b3n-a-la-espera-de-ser-recogidos-por-el-triturador-de-basura-reciclaje-de-papel-usado-.jpg
Extracting features for Imagenes depuradas/downloads/basura carton\10.chica-sosteniendo-caja-carton-papel-dibujos-animados-residuos-usados-basura-reciclaje_157402-2229.jpg
Extracting features for Imagenes depuradas/downloads/basura carton\11.reci3.jpg
Extracting features for Imagenes depuradas/downloads/basura carton\12.basura-de-las-cajas-de-cart%c3%b3n-48409511.jpg
Extracting features for Imagenes depuradas/downloads/basura carton\13.papel-cart%c3%b3n-reciclar.jpg
Extracting features for Imagenes depuradas/downloads/basura carton\14.residuos-papel-carton.jpg
Extracting features for Imagenes depuradas/downloads/basura carton\15.autdgrhdrfhwndklmvj3jwrljy.jpg
Extracting features for Imagenes depuradas/downloads/basu

Extracting features for Imagenes depuradas/downloads/basura de unicel\1.unicel_contenido1.jpg
Extracting features for Imagenes depuradas/downloads/basura de unicel\10.reciclar-unicel-d.jpg
Extracting features for Imagenes depuradas/downloads/basura de unicel\11.unicel.jpg
Extracting features for Imagenes depuradas/downloads/basura de unicel\14.discover-4612-styrofoam-larg_848x476_adaptiveresize.jpg
Extracting features for Imagenes depuradas/downloads/basura de unicel\15.reciclaje_unicel.jpg
Extracting features for Imagenes depuradas/downloads/basura de unicel\16.unicel_portada-1.jpg
Extracting features for Imagenes depuradas/downloads/basura de unicel\17.rs3099151_age-2018-09-03-21-22-42-lpr_crop1569449343650.jpg_554688468.jpg
Extracting features for Imagenes depuradas/downloads/basura de unicel\18.unicel.jpg
Extracting features for Imagenes depuradas/downloads/basura de unicel\19.p4-2-f1-6.jpg
Extracting features for Imagenes depuradas/downloads/basura de unicel\2.basura-unicel-1024x4

Extracting features for Imagenes depuradas/downloads/basura plastico\7.contaminacion_por_plastico-coca_cola-nestle_479463857_149609697_1024x576.jpg
Extracting features for Imagenes depuradas/downloads/basura plastico\8.image1440x560cropped.jpg
Extracting features for Imagenes depuradas/downloads/basura plastico\9.ogww5h535veqvji5bkhkm26jbu.jpg
Extracting features for Imagenes depuradas/downloads/basura ropa\1.fast-moda-desechos-ropa-3-1024x683.jpg
Extracting features for Imagenes depuradas/downloads/basura ropa\10.gettyimages-978477248.jpg
Extracting features for Imagenes depuradas/downloads/basura ropa\11.4256-e1474373288138.jpg
Extracting features for Imagenes depuradas/downloads/basura ropa\12.58.jpg
Extracting features for Imagenes depuradas/downloads/basura ropa\13.noticias-0x0-645088.jpg
Extracting features for Imagenes depuradas/downloads/basura ropa\14.1526929130_723098_1526929603_rrss_normal.jpg
Extracting features for Imagenes depuradas/downloads/basura ropa\15.ropa-1.jpg
Ext

Extracting features for Imagenes depuradas/downloads/bolsas de te usadas\19.reuse_old_tea_bags_featured.jpg
Extracting features for Imagenes depuradas/downloads/bolsas de te usadas\2.bolsas-de-te%cc%81.jpg
Extracting features for Imagenes depuradas/downloads/bolsas de te usadas\20.usos-de-las-bolsas-de-te-usadas.jpg
Extracting features for Imagenes depuradas/downloads/bolsas de te usadas\3.otros-usos-bolsas-de-te.jpg
Extracting features for Imagenes depuradas/downloads/bolsas de te usadas\4.usos-de-las-bolsas-de-te-usadas-4.jpg
Extracting features for Imagenes depuradas/downloads/bolsas de te usadas\5.que-hacer-con-las-bolsitas-de-t%c3%a9-usadas.jpg
Extracting features for Imagenes depuradas/downloads/bolsas de te usadas\6.7-usos-de-las-bolsas-de-te-usadas-en-el-jardin-01-e1475256867949.jpg
Extracting features for Imagenes depuradas/downloads/bolsas de te usadas\7.28072158-bolsas-de-t%c3%a9-usadas-con-etiqueta-en-blanco.jpg
Extracting features for Imagenes depuradas/downloads/bolsas de

Extracting features for Imagenes depuradas/downloads/chicles usados\20.arte+con+chicle.jpg
Extracting features for Imagenes depuradas/downloads/chicles usados\3.chicles.jpg
Extracting features for Imagenes depuradas/downloads/chicles usados\4.1447255161_420645_1447255417_album_normal.jpg
Extracting features for Imagenes depuradas/downloads/chicles usados\5.fondo-de-mucho-chicle-usado-99305505.jpg
Extracting features for Imagenes depuradas/downloads/chicles usados\6.zapatillas-2409-800x400.jpg
Extracting features for Imagenes depuradas/downloads/chicles usados\7._100183315_gumshoe.jpg
Extracting features for Imagenes depuradas/downloads/chicles usados\8.818gwuvi-dl._ac_sx466_.jpg
Extracting features for Imagenes depuradas/downloads/chicles usados\9.utilizado-mascar-plano-de-fondo-goma-retrato_csp50618375.jpg
Extracting features for Imagenes depuradas/downloads/colillas de cigarro\1.cigarros.jpg
Extracting features for Imagenes depuradas/downloads/colillas de cigarro\10.colillas.jpg
Extr

Extracting features for Imagenes depuradas/downloads/envases de tetrapack usados\4.en-mexico-desarrollan-maquina-recicladora-de-envases-de-tetra-pak-gr.jpg
Extracting features for Imagenes depuradas/downloads/envases de tetrapack usados\6.empaques-tetrapack-gr.jpg
Extracting features for Imagenes depuradas/downloads/envases de tetrapack usados\7.three_packages_feat_craft_effect_on_desk.jpg
Extracting features for Imagenes depuradas/downloads/envases de tetrapack usados\8.straw-hole-milk.jpg
Extracting features for Imagenes depuradas/downloads/envases de tetrapack usados\9.wp_000615b.jpg
Extracting features for Imagenes depuradas/downloads/envolturas metalicas\1.porque-las-envolturas-de-los-chocolates-son-de-aluminio.jpg
Extracting features for Imagenes depuradas/downloads/envolturas metalicas\10.72255998-textura-de-aluminio-arrugado-papel-de-aluminio-con-reflejos-de-luz-en-la-envoltura-met%c3%a1lica-de-plata-b.jpg
Extracting features for Imagenes depuradas/downloads/envolturas metalica

Extracting features for Imagenes depuradas/downloads/platos desechables usados\1.platos-desechables-no-son-una-opcion-para-los-eventos-0_ai1.jpg
Extracting features for Imagenes depuradas/downloads/platos desechables usados\10.getimage.jpg
Extracting features for Imagenes depuradas/downloads/platos desechables usados\11.elimina-desechables_9-887x1024.jpg
Extracting features for Imagenes depuradas/downloads/platos desechables usados\12.6a00d8341bfb1653ef019b002e7b5a970b-pi.jpg
Extracting features for Imagenes depuradas/downloads/platos desechables usados\13.71oyiftw5ql._ac_sx466_.jpg
Extracting features for Imagenes depuradas/downloads/platos desechables usados\14.vaso-desechable.jpg
Extracting features for Imagenes depuradas/downloads/platos desechables usados\15.shield-only-leaf-republic.jpg
Extracting features for Imagenes depuradas/downloads/platos desechables usados\16.plato-desechable.jpg
Extracting features for Imagenes depuradas/downloads/platos desechables usados\17.61u2xmoflyl

Extracting features for Imagenes depuradas/downloads/restos de pan\.ipynb_checkpoints\8.trozos-de-pan-y-restos-de-una-tabla-de-cortar-de-madera-kwaad3-checkpoint.jpg
Extracting features for Imagenes depuradas/downloads/restos de pan\.ipynb_checkpoints\9.encuentran-en-jordania-los-restos-de-pan-mas-antiguos-del-mundo-checkpoint.jpg
Extracting features for Imagenes depuradas/downloads/restos de tortillas\1.tortilla.jpg
Extracting features for Imagenes depuradas/downloads/restos de tortillas\10.tortilla_francesa_con_restos1.jpg
Extracting features for Imagenes depuradas/downloads/restos de tortillas\2.195113-87003.jpg
Extracting features for Imagenes depuradas/downloads/restos de tortillas\3.tortillas.jpg
Extracting features for Imagenes depuradas/downloads/restos de tortillas\4.tortillas-5247110_1920.jpg_346753400.jpg
Extracting features for Imagenes depuradas/downloads/restos de tortillas\5.como-hacer-tortillas-con-las-sobras-2.jpg
Extracting features for Imagenes depuradas/downloads/re

In [8]:
# To train the classifier 
class ClassifierTrainer(object): 
    def __init__(self, X, label_words): 
        # Encoding the labels (words to numbers) 
        self.le = preprocessing.LabelEncoder() 
 
        # Initialize One vs One Classifier using a linear kernel 
        self.clf = OneVsOneClassifier(LinearSVC(random_state=0)) 
 
        y = self._encodeLabels(label_words) 
        X = np.asarray(X) 
        self.clf.fit(X, y) 
 
    # Predict the output class for the input datapoint 
    def _fit(self, X): 
        X = np.asarray(X) 
        return self.clf.predict(X) 
 
    # Encode the labels (convert words to numbers) 
    def _encodeLabels(self, labels_words): 
        self.le.fit(labels_words) 
        return np.array(self.le.transform(labels_words), 
         dtype=np.float32) 
 
    # Classify the input datapoint 
    def classify(self, X): 
        labels_nums = self._fit(X) 
        labels_words = self.le.inverse_transform([int(x) for x in 
         labels_nums]) 
        return labels_words 

In [9]:
feature_map_file = 'feature_map_file'
svm_file = 'svm_file'

# Load the feature map 
with open(feature_map_file, 'rb') as f: 
    feature_map = pickle.load(f) 

# Extract feature vectors and the labels 
labels_words = [x['label'] for x in feature_map] 

# Here, 0 refers to the first element in the 
# feature_map, and 1 refers to the second 
# element in the shape vector of that element 
# (which gives us the size) 
dim_size = feature_map[0]['feature_vector'].shape[1] 

X = [np.reshape(x['feature_vector'], (dim_size,)) for x in feature_map] 

# Train the SVM 
svm = ClassifierTrainer(X, labels_words) 
with open(svm_file, 'wb') as f: 
    pickle.dump(svm, f) 

In [10]:
# Classifying an image 
class ImageClassifier(object): 
    def __init__(self, svm_file, codebook_file): 
        # Load the SVM classifier 
        with open(svm_file, 'rb') as f: 
            self.svm = pickle.load(f) 
 
        # Load the codebook 
        with open(codebook_file, 'rb') as f: 
            self.kmeans, self.centroids = pickle.load(f) 
 
    # Method to get the output image tag 
    def getImageTag(self, img): 
        # Resize the input image 
        img = resize_to_size(img) 
 
        # Extract the feature vector 
        feature_vector = FeatureExtractor().get_feature_vector(img, self.kmeans, self.centroids) 
 
        # Classify the feature vector and get the output tag 
        image_tag = self.svm.classify(feature_vector) 
 
        return image_tag 

In [14]:
svm_file = 'svm_file'
codebook_file = 'codebook_file'
input_image = cv2.imread('Imagenes depuradas/downloads/latas de conserva/2.-cuanto-pueden-durar-las-latas-de-conserva-1-620x350.jpg') 
 
tag = ImageClassifier(svm_file, codebook_file).getImageTag(input_image)
if ['Cascara de frutas'] in tag:
    print('Residuos orgánicos')
elif ['Cascara de verduras'] in tag:
    print('Residuos orgánicos')
elif ['Restos de Comida'] in tag:
    print('Residuos orgánicos')
elif ['Cascara de huevo'] in tag:
    print('Residuos orgánicos')
elif ['Restos de Pan'] in tag:
    print('Residuos orgánicos')
elif ['Restos de Tortilla'] in tag:
    print('Residuos orgánicos')
elif ['Filtro de cafe'] in tag:
    print('Residuos orgánicos')
elif ['Bolsas de te'] in tag:
    print('Residuos orgánicos')
elif ['Huesos'] in tag:
    print('Residuos orgánicos')
elif ['Desperdicio de pescado'] in tag:
    print('Residuos orgánicos')
elif ['Restos de Carne'] in tag:
    print('Residuos orgánicos')
elif ['Cascara de frutos secos'] in tag:
    print('Residuos orgánicos')
elif ['Servilletas'] in tag:
    print('Residuos inorgánicos no reciclables')
elif ['Unicel'] in tag:
    print('Residuos inorgánicos no reciclables')
elif ['Colillas de cigarros'] in tag:
    print('Residuos inorgánicos no reciclables')
elif ['Envoltorio de frituras'] in tag:
    print('Residuos inorgánicos no reciclables')
elif ['Envolturas metalicas'] in tag:
    print('Residuos inorgánicos no reciclables')
elif ['Chicles usados'] in tag:
    print('Residuos inorgánicos no reciclables')
elif ['Jeringas'] in tag:
    print('Residuos inorgánicos no reciclables')
elif ['Bolsas usadas'] in tag:
    print('Residuos inorgánicos no reciclables')
elif ['Vajillas de carton'] in tag:
    print('Residuos inorgánicos no reciclables')
elif ['Platos desechables'] in tag:
    print('Residuos inorgánicos no reciclables')
elif ['Papel'] in tag:
    print('Residuos inorgánicos reciclables')
elif ['Carton'] in tag:
    print('Residuos inorgánicos reciclables')
elif ['Vidrio'] in tag:
    print('Residuos inorgánicos reciclables')
elif ['Plastico'] in tag:
    print('Residuos inorgánicos reciclables')
elif ['Metal'] in tag:
    print('Residuos inorgánicos reciclables')
elif ['Ropa'] in tag:
    print('Residuos inorgánicos reciclables')
elif ['Tetrapack'] in tag:
    print('Residuos inorgánicos reciclables')
elif ['Latas de conserva'] in tag:
    print('Residuos inorgánicos reciclables')
elif ['Latas de refresco'] in tag:
    print('Residuos inorgánicos reciclables')
elif ['Recipientes PVC'] in tag:
    print('Residuos inorgánicos reciclables')
elif ['Madera'] in tag:
    print('Residuos inorgánicos reciclables')
else:
    print("Sepa la verga")

Residuos inorgánicos reciclables
